In [ ]:
# Install required packages
!pip install ultralytics albumentations rich



In [ ]:
# Clone your repository
!git clone https://github.com/your-username/LEGO_Bricks_ML_Vision.git
%cd LEGO_Bricks_ML_Vision


In [ ]:

# Run the training
!python lego_cli.py multiclass-demo-train --dataset C:\Users\User\Projects_Unprotected\LEGO_Bricks_ML_Vision\data\Multiclass_Bricks_dataset --epochs 100 --patience 5 